IAMSI -- 2017-2018
--------
*&copy;Equipe pédagogique IAMSI'2018*


# TME 09 : Règles d'association

<font size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

*BALDE Ahmed Tidiane*

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce fichier ipython complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un second envoi peut être fait **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyez le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

#### A télécharger

Avant de commencer, télécharger l'archive **tme-09.tgz** à l'adresse indiquée.

Une fois détarée, cette archive crée un répertoire contenant :
- deux fichiers exécutables (Linux): *apriori32* et *fpgrowth32* (ces 2 programmes ont été développés par Christian Borgelt, chercheur au ``European Center of Soft Computing" , plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance.
- un répertoire *datasets* contenant des bases d'exemples pour tester les algorithmes
- ce fichier ipython notebook que vous lisez en ce moment et que vous allez compléter.



### Imports utilisés

In [39]:
import os
import time
from functools import reduce

## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier "*exemple-1.txt*" (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [40]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font size="+1">**[Q]**</font> Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble d'items plutôt que comme une liste. Il sera ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [41]:
def chargeBase(fichier, delimiter=' '):
    """ Prends en argument un fichier et renvoie un dictionnaire 
        avec comme clé les numéros de ligne et les valeurs le set des itemsets
        
        :param fichier: fichier en entrée contenant des transactions
        :return: BASE
    """
    
    with open(fichier, 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=delimiter)
        return {i: set(itemsets) for i, itemsets in enumerate(lecteur)}

In [42]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font size="+1">**[Q]**</font> Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [43]:
def noms_items(BASE):
    """ Fonction qui prends en argument une BASE et rend 
        l'ensemble des items qui la compose 
        
        :param BASE: dict of sets
        :return: ensemble d'itemsets
    """
    return reduce((lambda x, y: x | y), BASE.values())

In [44]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font size="+1">**[Q]**</font> Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

Remarque: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [45]:
def singletons(BASE):
    return [{itemsets} for itemsets in noms_items(BASE)]

In [46]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'c'}, {'d'}, {'a'}, {'b'}, {'e'}]

<font size="+1">**[Q]**</font> Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [47]:
def comptage(BASE, itemset):
    comptage = 0
    res_boolean = [list(map(lambda x: x in transaction, itemset)) 
                   for transaction in BASE.values()]
    comptage = sum([reduce(lambda x, y: x and y, element) for element in res_boolean])
        
    return comptage

In [48]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font size="+1">**[Q]**</font> Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [49]:
def support(BASE, itemset):
    return comptage(BASE, itemset)/len(BASE.keys())

In [50]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : "+str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [51]:
def isCandidat(c, F):
    for i in c:
        if c-{i} not in F:
            return False
    return True

def apriori_gen(itemsets):
    res = []
    itemcopy = itemsets.copy()
    while (itemcopy):
        E = itemcopy.pop(0)
        gen = [E|l for l in itemcopy]
        for itemset in gen:
            if (len(itemset) == len(E)+1 
                and isCandidat(itemset ,itemsets) 
                and itemset not in res):
                res.append(itemset)
    return res

In [52]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))


Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'a', 'b'}, {'a', 'c'}, {'d', 'a'}, {'b', 'c'}, {'d', 'b'}, {'d', 'c'}]

Exemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend 
[{'d', 'a', 'b'}, {'d', 'b', 'c'}]


<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [53]:
def apriori(BASE, min_support):
    candidats = singletons(BASE)
    res = [(itemset, support(BASE, itemset)) for itemset in candidats]
    while candidats:
        candidats = apriori_gen(candidats)
        supports  = list(map(lambda itemset: support(BASE, itemset), candidats))
        c_s = list(filter(lambda c: c[1] >= min_support, zip(candidats, supports)))
        candidats, supports = zip(*c_s) if len(c_s) else ([], [])
        candidats = list(candidats)
        for c, s in c_s:
            res.append((c, s))
    return res

In [54]:
print("Exemple: apriori(Base1, 0.3]) rend ")
apriori(Base1,0.3)

Exemple: apriori(Base1, 0.3]) rend 


[({'c'}, 0.7),
 ({'d'}, 0.7),
 ({'a'}, 0.6),
 ({'b'}, 0.7),
 ({'e'}, 0.3),
 ({'c', 'd'}, 0.4),
 ({'a', 'c'}, 0.4),
 ({'b', 'c'}, 0.6),
 ({'a', 'd'}, 0.4),
 ({'b', 'd'}, 0.4),
 ({'d', 'e'}, 0.3),
 ({'a', 'b'}, 0.5),
 ({'b', 'c', 'd'}, 0.3),
 ({'a', 'b', 'c'}, 0.4),
 ({'a', 'b', 'd'}, 0.3)]

## Expérimentations

### Vérification avec la base du TD

Utilisez votre fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que vous obtenez les bons itemsets fréquents.

In [55]:
a, b, c, d, e = 'a', 'b', 'c', 'd', 'e'
BaseTD = {
    0: {a, b, d, e},
    1: {b, c, d},
    2: {a, b, d, e},
    3: {a, c, d, e},
    4: {b, c, d, e},
    5: {b, d, e},
    6: {c, d},
    7: {a, b, c},
    8: {a, d, e},
    9: {b, d}
}

In [56]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
apriori(BaseTD,0.3)

Exemple: apriori(BaseTD, 0.3) rend 


[({'c'}, 0.5),
 ({'d'}, 0.9),
 ({'a'}, 0.5),
 ({'b'}, 0.7),
 ({'e'}, 0.6),
 ({'c', 'd'}, 0.4),
 ({'b', 'c'}, 0.3),
 ({'a', 'd'}, 0.4),
 ({'b', 'd'}, 0.6),
 ({'d', 'e'}, 0.6),
 ({'a', 'b'}, 0.3),
 ({'a', 'e'}, 0.4),
 ({'b', 'e'}, 0.4),
 ({'a', 'd', 'e'}, 0.4),
 ({'b', 'd', 'e'}, 0.4)]

### Comparaisons avec les implémentations apriori32 et fpgrowth

<font size="+1">**[Q]**</font> Tester votre programme sur les 3 bases données dans datasets et confronter vos résultats avec le programme *apriori32*.

In [67]:
# Test of apriori32 on 'datasets/exemple-1'
os.system('./apriori32 -s30 datasets/exemple-1.txt fichier-resultat.out')
print(open('fichier-resultat.out', 'r').read())

e d (30)
e (30)
a (60)
a b (50)
a b c (40)
a b d (30)
a c (40)
a d (40)
b (70)
b c (60)
b c d (30)
b d (40)
c (70)
c d (40)
d (70)



In [59]:
# Test of our program on 'datasets/mushrooms.txt'
BaseMushrooms = chargeBase('datasets/mushrooms.txt', ',')
apriori(BaseMushrooms, .3)

[({'NONE'}, 0.45817490494296575),
 ({'FLAT'}, 0.3911596958174905),
 ({'BLACK'}, 0.2656844106463878),
 ({'FLARING'}, 0.005703422053231939),
 ({'SOLITARY'}, 0.20342205323193915),
 ({'CONVEX'}, 0.45104562737642584),
 ({'ATTACHED'}, 0.025665399239543727),
 ({'WOODS'}, 0.3754752851711027),
 ({'URBAN'}, 0.043726235741444866),
 ({'RED'}, 0.19320342205323193),
 ({'BROAD'}, 0.6986692015209125),
 ({'CREOSOTE'}, 0.022813688212927757),
 ({'FOUL'}, 0.25665399239543724),
 ({'WHITE'}, 0.9771863117870723),
 ({'CHOCOLATE'}, 0.22433460076045628),
 ({'EQUAL'}, 0.1634980988593156),
 ({'EVANESCENT'}, 0.36311787072243346),
 ({'SILKY'}, 0.3431558935361217),
 ({'FREE'}, 0.9743346007604563),
 ({'NARROW'}, 0.30133079847908745),
 ({'PATHS'}, 0.1359315589353612),
 ({'BRUISES'}, 0.4011406844106464),
 ({'FIBROUS'}, 0.37250475285171103),
 ({'BELL'}, 0.05370722433460076),
 ({'BROWN'}, 0.5881653992395437),
 ({'ANISE'}, 0.04752851711026616),
 ({'GRASSES'}, 0.28564638783269963),
 ({'TWO'}, 0.07129277566539924),
 ({'ORAN

In [69]:
# Test of apriori32 on 'datasets/mushrooms.txt'
os.system('./apriori32 -s30 datasets/mushrooms.txt fichier-mushrooms.out')
print(open('fichier-mushrooms.out', 'r').read())

PARTIAL (100)
NARROW PARTIAL ONE FREE WHITE (30.1331)
NARROW PARTIAL ONE FREE (30.1331)
NARROW PARTIAL ONE WHITE (30.1331)
NARROW PARTIAL ONE (30.1331)
NARROW PARTIAL FREE WHITE (30.1331)
NARROW PARTIAL FREE (30.1331)
NARROW PARTIAL WHITE (30.1331)
NARROW PARTIAL (30.1331)
NARROW ONE FREE WHITE (30.1331)
NARROW ONE FREE (30.1331)
NARROW ONE WHITE (30.1331)
NARROW ONE (30.1331)
NARROW FREE WHITE (30.1331)
NARROW FREE (30.1331)
NARROW WHITE (30.1331)
NARROW (30.1331)
BUFF PARTIAL CLOSE (31.654)
BUFF PARTIAL (31.654)
BUFF CLOSE (31.654)
BUFF (31.654)
BUFF POISONOUS PARTIAL CLOSE FREE WHITE (30.5133)
BUFF POISONOUS PARTIAL CLOSE FREE (30.5133)
BUFF POISONOUS PARTIAL CLOSE WHITE (30.5133)
BUFF POISONOUS PARTIAL CLOSE (30.5133)
BUFF POISONOUS PARTIAL FREE WHITE (30.5133)
BUFF POISONOUS PARTIAL FREE (30.5133)
BUFF POISONOUS PARTIAL WHITE (30.5133)
BUFF POISONOUS PARTIAL (30.5133)
BUFF POISONOUS CLOSE FREE WHITE (30.5133)
BUFF POISONOUS CLOSE FREE (30.5133)
BUFF POISONOUS CLOSE WHITE (30.5133)

In [70]:
# Test of our program on 'datasets/titanic-red.csv'
BaseTitanic = chargeBase('datasets/titanic-red.csv', delimiter=',')
apriori(BaseTitanic, .1)

[({'WestonSuperMareMooseJawSK'}, 0.001349527665317139),
 ({'BrennesNorwayNewYork'}, 0.001349527665317139),
 ({'RuotsinphytaaFinlandNewYorkNY'}, 0.005398110661268556),
 ({'LittleOnnHallStaffs'}, 0.002699055330634278),
 ({'1st'}, 0.3792172739541161),
 ({'DorchesterMA'}, 0.001349527665317139),
 ({'PhiladelphiaPA'}, 0.010796221322537112),
 ({'PomeroyWA'}, 0.001349527665317139),
 ({'AughnacliffCoLongfordIrelandNewYorkNY'}, 0.001349527665317139),
 ({'CoCorkIrelandCharlestownMA'}, 0.001349527665317139),
 ({'PerkinsCountySD'}, 0.001349527665317139),
 ({'pclass'}, 0.001349527665317139),
 ({'LosAngelesCA'}, 0.004048582995951417),
 ({'GuernseyEnglandEdgewoodRI'}, 0.001349527665317139),
 ({'IlfordEssexWinnipegMB'}, 0.004048582995951417),
 ({'HongKongNewYorkNY'}, 0.004048582995951417),
 ({'GlenRidgeNJ'}, 0.001349527665317139),
 ({'SouthingtonNoankCT'}, 0.002699055330634278),
 ({'ChicagoIL'}, 0.005398110661268556),
 ({'MilwaukeeWI'}, 0.004048582995951417),
 ({'LondonBrooklynNY'}, 0.00134952766531713

In [71]:
# Test of apriori32 on 'datasets/titanic-red.csv'
os.system('./apriori32 -s10 datasets/titanic-red.csv fichier-titanic.out')
print(open('fichier-titanic.out', 'r').read())

Cherbourg (23.7517)
Cherbourg 1st (16.5992)
Cherbourg 1st alive (11.336)
Cherbourg female (11.336)
Cherbourg female alive (10.3914)
Cherbourg alive (14.7099)
Cherbourg male (12.4157)
3rd (27.1255)
3rd dead (19.5682)
3rd dead male (15.5196)
3rd dead male Southampton (12.2807)
3rd dead Southampton (14.5749)
3rd male (18.3536)
3rd male Southampton (14.7099)
3rd Southampton (19.5682)
2nd (34.8178)
2nd female (14.17)
2nd female alive (12.4157)
2nd female alive Southampton (10.6613)
2nd female Southampton (12.4157)
2nd alive (15.7895)
2nd alive Southampton (13.3603)
2nd dead (19.0283)
2nd dead male (17.274)
2nd dead male Southampton (15.5196)
2nd dead Southampton (17.274)
2nd male (20.6478)
2nd male Southampton (18.2186)
2nd Southampton (30.6343)
1st (37.9217)
1st female (15.9244)
1st female alive (15.2497)
1st alive (23.0769)
1st alive Southampton (11.471)
1st dead (14.8448)
1st dead male (14.17)
1st male (21.9973)
1st male Southampton (13.3603)
1st Southampton (20.9177)
female (38.8664)
fe

<font size="+1">**[Q]**</font> Utiliser le programme *apriori32* pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [72]:
# Test of apriori32 for generating assoc rules
starttime = time.time()

os.system('./apriori32 -trs30c70m2n5 datasets/mushrooms.txt apriori32-assoc-rules-m.out')

duree = time.time() - starttime
print("Temps d'execution : {}\n=================".format(duree))
print(open('apriori32-assoc-rules-m.out').read())

Temps d'execution : 0.03657937049865723
? <- NARROW ONE FREE WHITE (30.1331, 71.2934)
? <- NARROW ONE FREE PARTIAL (30.1331, 71.2934)
? <- NARROW ONE FREE (30.1331, 71.2934)
? <- NARROW ONE WHITE PARTIAL (30.1331, 71.2934)
? <- NARROW ONE WHITE (30.1331, 71.2934)
? <- NARROW ONE PARTIAL (30.1331, 71.2934)
? <- NARROW ONE (30.1331, 71.2934)
? <- NARROW FREE WHITE PARTIAL (30.1331, 71.2934)
? <- NARROW FREE WHITE (30.1331, 71.2934)
? <- NARROW FREE PARTIAL (30.1331, 71.2934)
? <- NARROW FREE (30.1331, 71.2934)
? <- NARROW WHITE PARTIAL (30.1331, 71.2934)
? <- NARROW WHITE (30.1331, 71.2934)
? <- NARROW PARTIAL (30.1331, 71.2934)
? <- NARROW (30.1331, 71.2934)
? <- SEVERAL NO CLOSE ONE (31.1787, 71.6463)
? <- SEVERAL NO CLOSE FREE (30.038, 70.5696)
? <- SEVERAL NO CLOSE WHITE (30.038, 70.5696)
? <- SEVERAL NO CLOSE PARTIAL (31.1787, 71.6463)
? <- SEVERAL NO CLOSE (31.1787, 71.6463)
EVANESCENT <- NARROW ONE FREE WHITE (30.1331, 72.5552)
EVANESCENT <- NARROW ONE FREE PARTIAL (30.1331, 72.55

<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font size="+1">**[Q]**</font>  Utiliser *fpgrowth32* pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [73]:
# Test of fpgrowth32 for generating assoc rules
starttime = time.time()

os.system('./fpgrowth32 -trs30c70m2n5 datasets/mushrooms.txt fpgrowth32-assoc-rules-m.out')

duree = time.time() - starttime
print("Temps d'execution : {}\n=================".format(duree))
print(open('fpgrowth32-assoc-rules-m.out').read())

# On remarque que sur cette base de données, fpgrowth32 est bien plus rapide 
# lors de la génération de règles d'associations

Temps d'execution : 0.028604507446289062
PARTIAL <- WHITE (97.7186, 100)
WHITE <- PARTIAL (100, 97.7186)
PARTIAL <- FREE (97.4335, 100)
FREE <- PARTIAL (100, 97.4335)
PARTIAL <- FREE WHITE (97.4335, 100)
WHITE <- FREE PARTIAL (97.4335, 100)
FREE <- WHITE PARTIAL (97.7186, 99.7082)
WHITE <- FREE (97.4335, 100)
FREE <- WHITE (97.7186, 99.7082)
PARTIAL <- ONE (92.3004, 100)
ONE <- PARTIAL (100, 92.3004)
PARTIAL <- ONE WHITE (90.019, 100)
WHITE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- WHITE PARTIAL (97.7186, 92.1206)
WHITE <- ONE (92.3004, 97.5283)
ONE <- WHITE (97.7186, 92.1206)
PARTIAL <- ONE FREE (90.019, 100)
FREE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- FREE PARTIAL (97.4335, 92.3902)
PARTIAL <- ONE FREE WHITE (90.019, 100)
WHITE <- ONE FREE PARTIAL (90.019, 100)
FREE <- ONE WHITE PARTIAL (90.019, 100)
ONE <- FREE WHITE PARTIAL (97.4335, 92.3902)
WHITE <- ONE FREE (90.019, 100)
FREE <- ONE WHITE (90.019, 100)
ONE <- FREE WHITE (97.4335, 92.3902)
FREE <- ONE (92.3004, 97.5283)
ONE <- F

<font size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

In [74]:
# Test of fpgrowth32 for generating assoc rules
starttime = time.time()

os.system('./fpgrowth32 -trs30c70m2n5 datasets/titanic-red.csv fpgrowth32-assoc-rules-t.out')

duree = time.time() - starttime
print("Temps d'execution : {}\n=================".format(duree))
print(open('fpgrowth32-assoc-rules-t.out').read())

Temps d'execution : 0.005621671676635742
Southampton <- male (60.9987, 75.885)
Southampton <- dead (53.4413, 77.2727)
Southampton <- dead male (46.9636, 78.4483)
male <- dead Southampton (41.2955, 89.2157)
dead <- male Southampton (46.2888, 79.5918)
male <- dead (53.4413, 87.8788)
dead <- male (60.9987, 76.9912)
alive <- female (38.8664, 83.3333)
Southampton <- 2nd (34.8178, 87.9845)



## Annexes: utilisation des programmes *apriori32* et *fpgrowth32*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori32*, soit *fpgrowth32*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth32 -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
